In [86]:
from datetime import datetime, timedelta, time
import plotly.graph_objects as go
import pandas as pd
from azimuth import az

In [87]:
now = datetime.now()
start_date = datetime.combine(now.date(), time(now.hour)) - timedelta(days=7)
df = binance_df = az.crypto.candles('BTC/USDT', provider="binance", interval="1h", start_date=start_date).to_df()
df['value'] = df['value'].apply(lambda x: round(x, 2))

In [88]:
df

,date,open,high,low,close,volume,value
0,2024-09-30 00:00:00,65812.01,65973.16,65800.00,65868.60,206.23762,1.358400e+07
1,2024-09-30 01:00:00,65868.60,65948.86,65578.66,65674.42,593.42452,3.900553e+07
2,2024-09-30 02:00:00,65674.42,65731.55,65528.00,65602.01,449.67342,2.950312e+07
3,2024-09-30 03:00:00,65602.01,65618.80,65094.00,65463.85,1154.50096,7.542401e+07
4,2024-09-30 04:00:00,65463.86,65544.00,64699.98,64769.99,1424.06087,9.262703e+07
...,...,...,...,...,...,...,...
179,2024-10-07 11:00:00,63532.00,63706.00,63520.00,63554.62,529.85189,3.370282e+07
180,2024-10-07 12:00:00,63554.62,63554.63,63088.00,63210.83,1154.64233,7.303259e+07
181,2024-10-07 13:00:00,63210.82,63222.00,62865.09,62891.48,1095.22985,6.902523e+07
182,2024-10-07 14:00:00,62891.49,63179.99,62649.95,63135.44,1677.47151,1.054870e+08


In [89]:
chart = go.Figure(data=[go.Candlestick(x=df['date'], open=df['open'], high=df['high'], low=df['low'], close=df['close'])])
chart.update_layout(xaxis_rangeslider_visible=False)
chart.show()

In [90]:
df = mexc_df = az.crypto.candles('BTC/USDT', provider="mexc", interval="1h", start_date=start_date).to_df()
df

,date,open,high,low,close,volume,value
0,2024-09-30 00:00:00,65812.00,65973.17,65799.99,65868.60,238.243604,15691932.24
1,2024-09-30 01:00:00,65868.60,65948.98,65578.56,65674.41,454.562113,29889073.95
2,2024-09-30 02:00:00,65674.41,65731.55,65528.00,65602.01,300.354577,19709146.99
3,2024-09-30 03:00:00,65602.01,65618.80,65094.00,65463.85,713.760754,46655507.57
4,2024-09-30 04:00:00,65463.85,65544.00,64700.00,64769.99,632.281349,41196549.95
...,...,...,...,...,...,...,...
179,2024-10-07 11:00:00,63532.00,63706.01,63519.99,63554.63,504.897101,32116658.21
180,2024-10-07 12:00:00,63554.63,63554.63,63088.00,63210.83,693.343581,43853014.14
181,2024-10-07 13:00:00,63210.83,63221.94,62877.78,62891.48,490.970849,30947634.89
182,2024-10-07 14:00:00,62891.48,63161.18,62649.97,63135.42,918.724559,57755954.94


In [91]:
chart = go.Figure(data=[go.Candlestick(x=df['date'], open=df['open'], high=df['high'], low=df['low'], close=df['close'])])
chart.update_layout(xaxis_rangeslider_visible=False)
chart.show()

In [98]:
assert len(binance_df) == len(mexc_df)
df = pd.DataFrame({
    'date': binance_df['date'],
    'b_avg_price': (binance_df['value'] / binance_df['volume']).round(2),
    'm_avg_price': (mexc_df['value'] / mexc_df['volume']).round(2),
    'spreed': (binance_df['value'] / binance_df['volume'] - mexc_df['value'] / mexc_df['volume']).round(2),
    'b_volume': binance_df['volume'],
    'm_volume': mexc_df['volume']
    
})
df.assign(spreed_val=(df.spreed * df.m_volume).where(df.spreed > 0, df.spreed * df.b_volume * (-1))).round(2)

,date,b_avg_price,m_avg_price,spreed,b_volume,m_volume,spreed_val
0,2024-09-30 00:00:00,65865.78,65865.07,0.71,206.24,238.24,169.15
1,2024-09-30 01:00:00,65729.55,65753.55,-24.00,593.42,454.56,14242.19
2,2024-09-30 02:00:00,65610.11,65619.60,-9.49,449.67,300.35,4267.40
3,2024-09-30 03:00:00,65330.40,65365.75,-35.35,1154.50,713.76,40811.61
4,2024-09-30 04:00:00,65044.29,65155.41,-111.12,1424.06,632.28,158241.64
...,...,...,...,...,...,...,...
179,2024-10-07 11:00:00,63608.01,63610.30,-2.30,529.85,504.90,1218.66
180,2024-10-07 12:00:00,63251.26,63248.60,2.66,1154.64,693.34,1844.29
181,2024-10-07 13:00:00,63023.51,63033.55,-10.04,1095.23,490.97,10996.11
182,2024-10-07 14:00:00,62884.54,62865.37,19.17,1677.47,918.72,17611.95
